In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, MaxPooling1D, Conv1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')
import pickle
vocab_size = 10000
batch_size = 32
epochs = 6
max_len = 25

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IITR/TheDataF.csv')
df2 = pd.read_csv('/content/drive/MyDrive/IITR/test_data.csv')
df=df.drop(df.columns[0], axis=1)
df=df.drop(df.columns[-1], axis=1)
df=df.rename(columns={"text":"comment_text"})
df2=df2.drop(df2.columns[0], axis=1)
df2=df2.rename(columns={"text":"comment_text"})
df_toxic = df.drop(['id', 'comment_text'], axis=1)
categories = list(df_toxic.columns.values)
train_df = df
test_df = df2

In [ ]:
df=train_df
df2=test_df


def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text
import nltk
#nltk.download()
from nltk.corpus import stopwords
import re
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
df['comment_text'] = df['comment_text'].str.lower()
df['comment_text'] = df['comment_text'].apply(cleanHtml)
df['comment_text'] = df['comment_text'].apply(cleanPunc)
df['comment_text'] = df['comment_text'].apply(keepAlpha)
df2['comment_text'] = df2['comment_text'].str.lower()
df2['comment_text'] = df2['comment_text'].apply(cleanHtml)
df2['comment_text'] = df2['comment_text'].apply(cleanPunc)
df2['comment_text'] = df2['comment_text'].apply(keepAlpha)

stop_words = set(stopwords.words('english'))
#stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
df['comment_text'] = df['comment_text'].apply(removeStopWords)
df2['comment_text'] = df2['comment_text'].apply(removeStopWords)

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatization(sentence):
    lemmSentence = ""
    for word in sentence.split():
        lemm = lemmatizer.lemmatize(word)
        lemmSentence += lemm
        lemmSentence += " "
    lemmSentence = lemmSentence.strip()
    return lemmSentence
df['comment_text'] = df['comment_text'].apply(lemmatization)
df2['comment_text'] = df2['comment_text'].apply(lemmatization)

df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))
df2['comment_text'] = df2['comment_text'].map(lambda com : clean_text(com))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#train_df['comment_text'] = train_df['comment_text'].apply(lambda x: clean_text(x))
#test_df['comment_text'] = test_df['comment_text'].apply(lambda x: clean_text(x))

In [ ]:
import tensorflow as tf
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='accuracy', 
                                              patience=3, verbose=0, 
                                              restore_best_weights=True)]

for category in categories:
    
    ##Trying something different
    '''
    df3 = train_df.copy()
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    rows=train_df.shape[0]
    row=609
    while (row<df3.shape[0]):
      if df3[category].iloc[row]==0:
        df3=df3.drop([df3.index[row]])
      else:
        row=row+1
    train=df3
    x_train, y_train = train['comment_text'].iloc[:].values, train[category].iloc[:].values
    ## Tried something different
    '''
    print('... Processing {}'.format(category))
    x_train, y_train = train_df['comment_text'].iloc[:].values, train_df[category].iloc[:].values
    print("\n",x_train.shape,"\n")

    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(x_train)
    sequences = tokenizer.texts_to_sequences(x_train)
    x_train_new = pad_sequences(sequences, maxlen=max_len)
    
    model = Sequential()
    model.add(Embedding(vocab_size, 128))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    #model.add(LSTM(128, dropout=0.2))
    model.add(Bidirectional(LSTM(128, dropout=0.5)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(64, dropout=0.2))
    #model.add(Dense(64, activation='tanh'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

    model.fit(x_train_new, y_train, batch_size=batch_size, epochs=epochs, verbose = 1, callbacks=callbacks)
    
    
    
    
    x_test = test_df['comment_text'].iloc[:].values
    sequences = tokenizer.texts_to_sequences(x_test)
    x_test_new = pad_sequences(sequences, maxlen=max_len)
    test_df[category] = model.predict(x_test_new)
    '''
    filename = '/content/Saved_ModelName calling/'+ category.replace("/",",").replace("'", "") + '.h5'
    model.save(filename)  # creates a HDF5 file
    '''
    del model  # deletes the existing model
    
    # returns a compiled model
    # identical to the previous one
#   model = load_model('my_model.h5')


... Processing  'Bandwagon'

 (7927,) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 32)          12320     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               164864    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense

In [ ]:
df2["labels"]=""
print(df2.shape)
for row in df2.index:
    dict2=[]
    for category in categories:
        if (df2[category][row]>=0.5):
            dict2.append(category[1:-1])
            #print("hello")
            #df2["labels"][row]+=pil
    dict2.sort()
    df2['labels'][row]=dict2
#df2["labels"]=df2["labels"].str.rstrip(',')
#df2["labels"]+=']'
submission = df2[['id','comment_text','labels']]
submission.head()
out = submission.to_json(orient='records', indent=4)
with open('temp.txt', 'w') as f:
    f.write(out)

(63, 23)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
categories

[" 'Bandwagon'",
 '"Misrepresentation of Someone\'s Position (Straw Man)"',
 "'Appeal to authority'",
 "'Appeal to fear/prejudice'",
 "'Black-and-white Fallacy/Dictatorship'",
 "'Causal Oversimplification'",
 "'Doubt'",
 "'Exaggeration/Minimisation'",
 "'Flag-waving'",
 "'Glittering generalities (Virtue)'",
 "'Loaded Language'",
 "'Name calling/Labeling'",
 'Obfuscation, Intentional vagueness, Confusion',
 "'Presenting Irrelevant Data (Red Herring)'",
 "'Reductio ad hitlerum'",
 "'Repetition'",
 "'Slogans'",
 "'Smears'",
 "'Thought-terminating cliché'",
 "'Whataboutism'"]